In [1]:
from stats_nba_web_scrapper import * 
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [2]:
path_API_download =  "/Users/rishabhpatni/Desktop/Projects/nba-play-by-play-probability-generator/final_values.csv"
path_defender_directory = "/Users/rishabhpatni/Desktop/Projects/nba-play-by-play-probability-generator/Data/"

In [3]:
dataset = PlayByPlayNBAApiDataset(defender_distance_data = path_defender_directory ,file_path = path_API_download).data.reset_index(drop=True)

177818


## Modelling

In [4]:
dataset.head(2)

,player_name,team_name,period,time,shot_zone,shot_distance,opponent,x,y,dribble_range,touch_time,closest_def_dist,value,made,margin,shot_type
0,Bam Adebayo,Miami Heat,1,10.0,Center(C),17,NOP,-14,171,0 Dribbles,Touch < 2 Seconds,6+ Feet - Wide Open,2.0,False,0.0,LongMidRange
1,Bam Adebayo,Miami Heat,1,8.0,Center(C),6,NOP,-57,34,0 Dribbles,Touch < 2 Seconds,2-4 Feet - Tight,2.0,False,-4.0,ShortMidRange


In [5]:
three_flag = pd.get_dummies(dataset["value"].apply(lambda x: 1 if x == 2.0 else 0), drop_first = True)
player_name = pd.get_dummies(dataset["player_name"], drop_first=True)
team_name = pd.get_dummies(dataset["team_name"], drop_first=True)
shot_zone = pd.get_dummies(dataset["shot_zone"], drop_first=True)
opponent = pd.get_dummies(dataset["opponent"], drop_first=True)
dribble_range = pd.get_dummies(dataset["dribble_range"], drop_first=True)
touch_time = pd.get_dummies(dataset["touch_time"], drop_first=True)
closest_def_dist = pd.get_dummies(dataset["closest_def_dist"], drop_first=True)
shot_type = pd.get_dummies(dataset["shot_type"], drop_first=True)


dataset.drop(["player_name", "team_name", "shot_zone", "opponent", 
              "dribble_range", "touch_time", "closest_def_dist", "value", "shot_type"], axis = 1, inplace = True)

dataset = pd.concat([dataset, player_name, team_name, shot_zone, opponent, 
              dribble_range, touch_time, closest_def_dist, three_flag, shot_type], axis =1)
dataset = dataset.apply(pd.to_numeric)

In [6]:
logistic_instance = LogisticRegression()

In [7]:
dataset.index

RangeIndex(start=0, stop=71750, step=1)

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataset.drop("made", axis = 1),dataset.made,  test_size=0.25, random_state=0)

In [14]:
logistic_instance.fit(x_train, y_train)
prediction_model = logistic_instance.predict(x_test)
print(classification_report(y_test, prediction_model))

              precision    recall  f1-score   support

       False       0.61      0.84      0.70      9866
        True       0.63      0.33      0.43      8072

    accuracy                           0.61     17938
   macro avg       0.62      0.58      0.57     17938
weighted avg       0.61      0.61      0.58     17938



In [28]:
dataset_ref = PlayByPlayNBAApiDataset(defender_distance_data = path_defender_directory ,file_path = path_API_download).data.reset_index(drop=True)

177818


In [56]:
dataset_ref = dataset_ref[dataset_ref.index.isin(x_test.index)]
dataset_ref["prob"] = logistic_instance.predict_proba(x_test.sort_index())[:,1]
dataset_ref.head()

/Users/rishabhpatni/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/var/folders/7q/y6lh4q3x7l986nswgbdchpjw0000gn/T/ipykernel_2026/498356997.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_ref["prob"] = logistic_instance.predict_proba(x_test.sort_index())[:,1]


,player_name,team_name,period,time,shot_zone,shot_distance,opponent,x,y,dribble_range,touch_time,closest_def_dist,value,made,margin,shot_type,prob
3,Bam Adebayo,Miami Heat,2,10.0,Center(C),2,NOP,-13,16,0 Dribbles,Touch < 2 Seconds,2-4 Feet - Tight,2.0,True,6.0,AtRim,0.637674
14,Bam Adebayo,Miami Heat,1,9.0,Right Side Center(RC),16,MIL,69,147,0 Dribbles,Touch < 2 Seconds,6+ Feet - Wide Open,2.0,False,1.0,LongMidRange,0.488386
17,Bam Adebayo,Miami Heat,3,7.0,Center(C),15,MIL,30,151,0 Dribbles,Touch < 2 Seconds,6+ Feet - Wide Open,2.0,True,-14.0,LongMidRange,0.494798
18,Bam Adebayo,Miami Heat,4,7.0,Center(C),2,MIL,19,16,0 Dribbles,Touch < 2 Seconds,4-6 Feet - Open,2.0,False,7.0,AtRim,0.693876
24,Bam Adebayo,Miami Heat,2,3.0,Center(C),12,OKC,13,127,0 Dribbles,Touch < 2 Seconds,2-4 Feet - Tight,2.0,True,9.0,ShortMidRange,0.365665
